In [26]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()


In [27]:
import pandas as pd
file_path = 'spotify_songs.csv'
sales_data = pd.read_csv(file_path, nrows=20, encoding='latin1')

sales_data.head()


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [49]:
# Create assistant and conversation

file = client.files.create(
    file=open(file_path, "rb"),
    purpose='assistants',
)

assistant = client.beta.assistants.create(
    instructions="As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.",
    model="gpt-4-0125-preview",
    tools=[
        {"type": "code_interpreter"}
    ],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id]  # Here we add the file id
        }
    }
)

print(assistant)


Assistant(id='asst_Bjn3pRKRhvud7vdpIp7Qtyau', created_at=1748930250, description=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', metadata={}, model='gpt-4-0125-preview', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-YUg5s52u2ML91PQvf8qgGW']), file_search=None), top_p=1.0, reasoning_effort=None)


In [50]:
# **thread** can start a conversation

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": (
                "Using the uploaded dataset, create **three separate and clearly labeled charts** that analyze song trends over time.\n\n"
                "Chart 1: Scatter plot of Release Date vs. Popularity, colored by Artist (limit to top 10 most frequent artists).\n"
                "Chart 2: Bar chart showing the number of top songs by Genre.\n"
                "Chart 3: Line chart of average song popularity per year.\n\n"
                "Each chart should be saved as a **separate image**. Label axes and titles clearly. Do NOT combine them into a single figure."
            ),
            "attachments": [
                {
                    "file_id": file.id,
                    "tools": [{ "type": "code_interpreter" }]
                }
            ]
        }
    ]
)

print(thread)


Thread(id='thread_71Ptmllbedq69VLONcxu2v5t', created_at=1748930251, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-YUg5s52u2ML91PQvf8qgGW']), file_search=None))


In [51]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
print(run)


Run(id='run_3SJUUpKA0SVXcDYLhaR1aodG', assistant_id='asst_Bjn3pRKRhvud7vdpIp7Qtyau', cancelled_at=None, completed_at=None, created_at=1748930254, expires_at=1748930854, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_71Ptmllbedq69VLONcxu2v5t', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)


In [52]:
import time
while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        messages.data[0].content[0].image_file
        time.sleep(5)
        print('Chart created!')
        if messages.data and messages.data[0].content:
            print('Current Message:', messages.data[0].content[0])
        break
    except:
        time.sleep(10)
        print('Your Assistant is working on creating the chart...')
        if messages.data and messages.data[0].content:
            print('Current Message:', messages.data[0].content[0])


Your Assistant is working on creating the chart...
Current Message: TextContentBlock(text=Text(annotations=[], value='Using the uploaded dataset, create **three separate and clearly labeled charts** that analyze song trends over time.\n\nChart 1: Scatter plot of Release Date vs. Popularity, colored by Artist (limit to top 10 most frequent artists).\nChart 2: Bar chart showing the number of top songs by Genre.\nChart 3: Line chart of average song popularity per year.\n\nEach chart should be saved as a **separate image**. Label axes and titles clearly. Do NOT combine them into a single figure.'), type='text')
Your Assistant is working on creating the chart...
Current Message: TextContentBlock(text=Text(annotations=[], value="First, let's load and inspect the contents of the dataset to understand its structure. Once we have a better understanding, we can proceed with creating the requested charts."), type='text')
Your Assistant is working on creating the chart...
Current Message: TextCont

In [70]:
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)


plot_file_ids = [] # Also extracting plot file ids
# Saving the plots
for i in range(len(messages.data)):
    try:
        plot_file_id = messages.data[i].content[0].image_file.file_id
        image_path = "music trend analysis_{}.png".format(i)
        convert_file_to_png(plot_file_id, image_path)

        plot_file = client.files.create(
            file=open(image_path, "rb"),
            purpose='assistants'
        )

        plot_file_ids.append(plot_file.id)
    except:
        continue
        

In [54]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]


[ImageFileContentBlock(image_file=ImageFile(file_id='file-ENL6vQAiFECDpgcWybnUPN', detail=None), type='image_file'),
 ImageFileContentBlock(image_file=ImageFile(file_id='file-1PvwnBGQX8WnT2fFJmCkEH', detail=None), type='image_file'),
 ImageFileContentBlock(image_file=ImageFile(file_id='file-WBRfVdnazMNdZxrYjy9ypZ', detail=None), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value="The dataset contains various information about songs, including track name, artist, popularity, album information, release date, genre, and various audio features like key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, and duration.\n\nWith this information in hand, I will proceed to create the requested charts:\n\n1. **Scatter plot of Release Date vs. Popularity**, colored by the **Artist** (limited to the top 10 most frequent artists).\n2. **Bar chart** showing the **number of top songs by Genre**.\n3. **Line chart** of **average song popularity per

In [61]:
# AI-generated data insights

def submit_message_wait_completion(assistant_id, thread, user_message, file_ids=None):
    for run in client.beta.threads.runs.list(thread_id=thread.id).data:
        if run.status == "in_progress":
            print(f"Waiting for Run {run.id} to complete...")
            while True:
                run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
                if run_status in ['succeeded', 'failed']:
                    break
                time.sleep(5)

    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }

    if file_ids:
        attachments = [{"file_id": file_id, "tools": [ { "type": "code_interpreter" } ]} for file_id in file_ids]
        params['attachments'] = attachments

    client.beta.threads.messages.create(**params)

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    return run


In [65]:
submit_message_wait_completion(
    assistant.id,
    thread,
    "Based on the multiple charts you just created, for each chart, give me two bullet points of about 20 words each, describing the most important insights. This will be used in a slide presentation to reveal the 'secrets' behind the data."
)


Run(id='run_MMS2s8JLZevkVn6y9oMdmMZ8', assistant_id='asst_Bjn3pRKRhvud7vdpIp7Qtyau', cancelled_at=None, completed_at=None, created_at=1748930879, expires_at=1748931479, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_71Ptmllbedq69VLONcxu2v5t', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [66]:
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)


**Scatter Plot of Release Date vs. Popularity for Top 10 Artists:**
- Latest tracks by top artists show a varied range of popularity scores, without a clear correlation to release dates.
- Some artists consistently release highly popular tracks, indicating a strong and steady fan base or successful marketing approaches.

**Number of Top Songs by Genre Bar Chart:**
- Pop and dance genres lead in top songs, illustrating their broader popularity and cultural influence in contemporary music.
- Hip-hop, despite its significant cultural impact, has fewer top songs, suggesting genre-specific audience preferences or platform biases.

**Average Song Popularity Per Year Line Chart:**
- There’s a noticeable upward trend in song popularity, particularly from 2018, possibly driven by changes in streaming or consumption habits.
- The increasing popularity scores may also indicate a growing concentration of listens among newer songs or enhanced discovery via digital platforms.


In [67]:
submit_message_wait_completion(
    assistant.id,
    thread,
    "Based on the plots and key points you created, come up with a very short title for each slide. It should only reflect the main insights you have drawn."
)


Run(id='run_Hxmq9ZZUKHtS0C9kAwcqfhk7', assistant_id='asst_Bjn3pRKRhvud7vdpIp7Qtyau', cancelled_at=None, completed_at=None, created_at=1748930915, expires_at=1748931515, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_71Ptmllbedq69VLONcxu2v5t', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [68]:
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)


**Scatter Plot of Release Date vs. Popularity for Top 10 Artists:**
- "Trendsetters: The Impact of Artist and Time on Song Popularity"

**Number of Top Songs by Genre Bar Chart:**
- "Genre Giants: Pop and Dance Lead the Charts"

**Average Song Popularity Per Year Line Chart:**
- "Evolving Tastes: The Rise of Song Popularity Over Years"


In [69]:
company_summary = "We are a CD wholesaler, but our chairman is also a pop singer!"

response = client.images.generate(
    model="dall-e-3",
    prompt=f"Based on this company summary {company_summary}, create an inspirational photo showcasing growth and the path forward. This will be used for the quarterly sales planning meeting.",
    size="1024x1024",
    quality="hd",
    n=1
)

image_url = response.data[0].url

import requests
dalle_img_path = 'Music Trend Chart.png'
img = requests.get(image_url)

with open(dalle_img_path, 'wb') as file:
    file.write(img.content)

dalle_file = client.files.create(
    file=open(dalle_img_path, "rb"),
    purpose='assistants'
)


In [75]:
# Create the final PPT

title_template = """


from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

prs = Presentation()

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set slide background to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left 60% of the slide
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3 / 5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title on the right side
left = prs.slide_width * 3 / 5
top = Inches(2)
width = prs.slide_width * 2 / 5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.clear()
title_p = title_frame.paragraphs[0]
title_p.text = title_text
title_p.font.size = Pt(38)
title_p.font.bold = True
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle slightly lower on the right
top = Inches(3.2)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_frame.clear()
subtitle_p = subtitle_frame.paragraphs[0]
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(24)
subtitle_p.font.color.rgb = RGBColor(200, 200, 200)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

"""



data_vis_template = """


from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

prs = Presentation()

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales and Direct Sales Optimization"
bullet_points = "• Online sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct sales show fluctuations, indicating variability in performance and the need for targeted improvements."

left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3 / 5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

left = prs.slide_width * 2 / 3
top = Inches(1.5)
width = prs.slide_width * 1 / 3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()

bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

title_text = "Birdsong Paradise"
subtitle_text = "Pop Music Analysis"


submit_message_wait_completion(
    assistant.id,
    thread,
    f"""You are given three PNG charts that analyze trends in song popularity. 
Each chart corresponds to a different aspect of the dataset: (1) popularity over time, (2) distribution by genre, and (3) top 10 artist trends.

Create a PPTX file with:
- A **first title slide** using the inspirational image included (use it as `image_path`), title `{title_text}`, and subtitle `{subtitle_text}`.
- Then, for **each of the three attached charts**, generate **one slide** using the following template (used as `data_vis_template`):
{data_vis_template}

Each visualization slide should include:
- The image (`image_path`),
- An appropriate title you generate based on the chart contents (`title_text`),
- Two concise bullet insights (`bullet_points`) that summarize the trends in the chart.

Make sure the background is black as shown in the template. Output a single PPTX file with a total of 4 slides.

""",
    file_ids=[dalle_file.id, *plot_file_ids]
)

while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Your Assistant is working hard on creating the slides...")
        time.sleep(10)


import io
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file = client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("Promotional_Conference.pptx", "wb") as f:
    f.write(file_obj.getbuffer())




Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Successfully retrieved pptx_id: file-3hEU4xVHPgyQZfxnvaDS8g


**HW: from data analysis to ppt generation (your own business)**